# Anomaly detection using a local normalization

In [8]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/esa-esdl/ESDL.jl"))
Pkg.add(PackageSpec(url="https://github.com/esa-esdl/ESDLPlots.jl"))

    Updating git-repo `https://github.com/esa-esdl/ESDL.jl`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
    Updating git-repo `https://github.com/esa-esdl/ESDLPlots.jl`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [1]:
using ESDL

In [2]:
c=esdc() #Cube()
cdata=subsetcube( c, region="Sweden",
    variable=["air_temperature_2m","gross_primary_productivity","latent_energy","precipitation"], 
    time=2001:2008
)

YAXArray with the following dimensions
lon                 Axis with 53 Elements from 11.125 to 24.125
lat                 Axis with 55 Elements from 68.875 to 55.375
time                Axis with 368 Elements from 2001-01-05T00:00:00 to 2008-12-30T00:00:00
Variable            Axis with 4 elements: air_temperature_2m gross_primary_productivity latent_energy precipitation 
units: W m-2
Total size: 16.37 MB


In [3]:
cdata_filled=gapFillMSC(cdata)

YAXArray with the following dimensions
time                Axis with 368 Elements from 2001-01-05T00:00:00 to 2008-12-30T00:00:00
lon                 Axis with 53 Elements from 11.125 to 24.125
lat                 Axis with 55 Elements from 68.875 to 55.375
Variable            Axis with 4 elements: air_temperature_2m gross_primary_productivity latent_energy precipitation 
Total size: 16.37 MB


In [4]:
cdata_anom=removeMSC(cdata)

YAXArray with the following dimensions
time                Axis with 368 Elements from 2001-01-05T00:00:00 to 2008-12-30T00:00:00
lon                 Axis with 53 Elements from 11.125 to 24.125
lat                 Axis with 55 Elements from 68.875 to 55.375
Variable            Axis with 4 elements: air_temperature_2m gross_primary_productivity latent_energy precipitation 
Total size: 16.37 MB


In [5]:
cdata_norm=normalizeTS(cdata_anom)

YAXArray with the following dimensions
time                Axis with 368 Elements from 2001-01-05T00:00:00 to 2008-12-30T00:00:00
lon                 Axis with 53 Elements from 11.125 to 24.125
lat                 Axis with 55 Elements from 68.875 to 55.375
Variable            Axis with 4 elements: air_temperature_2m gross_primary_productivity latent_energy precipitation 
Total size: 16.37 MB


In [9]:
Pkg.add("MultivariateAnomalies")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [10]:
using MultivariateAnomalies
function cubeanomalies(data;methods=["REC","KDE"])
    any(ismissing,data) && return missing
    data = Float64.(data)
    o = hcat(detectAnomalies(data, methods)...)
    o
end

cubeanomalies (generic function with 1 method)

---- works until here ------

In [11]:
ooc = mapslices(cubeanomalies,cdata_norm,dims = ("time","variable"))

LoadError: UndefVarError: pairwise! not defined

In [ ]:
renameaxis!(ooc,"outaxis"=>CategoricalAxis("Method",["REC","KDE"]))

In [ ]:
using ESDLPlots
plotTS(ooc)

In [ ]:
display(plotMAP(ooc,time=Date(2003,7,25),method="REC"))
display(plotMAP(ooc,time=Date(2003,7,25),method="KDE",dmin=0.1,dmax=0.4))